<a href="https://colab.research.google.com/github/ucfilho/AI/blob/main/AI_2023/CV_stat_math/class_10_Automatic_Number_Plate_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INSTALLING ALL THE REQUIRED LIBRARIES**

In [1]:
!pip install easyocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install imutils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# IMPORTING ALL THE REQUIRED LIBRARIES

In [3]:
import cv2
import pandas as pd
import zipfile
import matplotlib.pyplot as plt
import numpy as np
import imutils 
from google.colab.patches import cv2_imshow
import easyocr

# Reading the Image

In [4]:
!gdown 17DeutH_Rzmw7J6Qdr9XdMFTRSH8v7VvY

Downloading...
From: https://drive.google.com/uc?id=17DeutH_Rzmw7J6Qdr9XdMFTRSH8v7VvY
To: /content/licence_plate.zip
100% 2.96M/2.96M [00:00<00:00, 212MB/s]


In [5]:
FILE = 'licence_plate.zip'
archive = zipfile.ZipFile(FILE, 'r')
names = archive.namelist()
archive.extractall()

In [6]:
#print(names)

In [7]:
licences = ['BDM3D69','POX4G21','BAH4952','K81482',
            '53JGZ5','6EVY261','BBRB364','FAG643',
            'C26727','GCP332', 'PRL3D97']
cars = ['placa_carro3.jpg', 'placa_carro2.jpg', '00011.jpg', 
        '00024.jpg', '00029.jpg', 'car_0001.jpg', 
        'car_0002.jpg', 'car_0004.jpg', 'car_0005.jpg', 
        'car_0006.jpg', 'placa_carro1.png']

# Converting the Image into the GRAY Scale FORMAT

In [8]:
def cropped(img):
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  bfilter = cv2.bilateralFilter(gray, 11, 11, 17)
  edged = cv2.Canny(bfilter, 30, 200)
  keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  contours = imutils.grab_contours(keypoints)
  contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]
  location = []
  for contour in contours:
    # cv2.approxPolyDP returns a resampled contour, so this will still return a set of (x, y) points
    approx = cv2.approxPolyDP(contour, 10, True)
    if len(approx) == 4:
      location = approx
      break
  mask = np.zeros(gray.shape, np.uint8)
  if len(location) > 0:
    new_image = cv2.drawContours(mask, [location], 0, 255, -1)

    new_image = cv2.bitwise_and(img, img, mask = mask)
    (x, y) = np.where(mask == 255)

    (x1, y1) = (np.min(x), np.min(y))

    (x2, y2) = (np.max(x), np.max(y))

    cropped_image = gray[x1:x2+3, y1:y2+3]

  else:
    new_image = gray
    cropped_image = gray
  
  return cropped_image

In [9]:
reader = easyocr.Reader(['en'])
font = cv2.FONT_HERSHEY_SIMPLEX
ans_all = []
ans_check =[]
for word in names:
  img = cv2.imread(word)
  scale_percent = 80 # percent of original size
  width = int(img.shape[1] * scale_percent / 100)
  height = int(img.shape[0] * scale_percent / 100)
  dim = (width, height)
  resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
  cropped_image = cropped(resized)
  result = reader.readtext(cropped_image)
  ans_check.append(result)
  licence_plate =""
  for word_ in result:
    w = word_[1]
    licence_plate += w + " "
  text = licence_plate
  idx = cars.index(word)
  ans_all.append([word,text,licences[idx]])


In [10]:
df =pd.DataFrame(ans_all,columns=['img','found','expected'] )

In [11]:
df

,img,found,expected
0,placa_carro3.jpg,C A PT U R,BDM3D69
1,placa_carro2.jpg,BR POXG21,POX4G21
2,00011.jpg,,BAH4952
3,00024.jpg,,K81482
4,00029.jpg,53JGZ-5;,53JGZ5
5,car_0001.jpg,GEv2al,6EVY261
6,car_0002.jpg,,BBRB364
7,car_0004.jpg,Porschd Potr n FAG 64,FAG643
8,car_0005.jpg,C26727,C26727
9,car_0006.jpg,,GCP332


In [12]:
ans_check

[[([[56, 4], [111, 4], [111, 23], [56, 23]], 'C A', 0.5810978412628174),
  ([[117, 5], [231, 5], [231, 21], [117, 21]], 'PT U R', 0.1402347837605933)],
 [([[2, 72], [28, 72], [28, 96], [2, 96]], 'BR', 0.9969396577231775),
  ([[21, 0], [309, 0], [309, 96], [21, 96]], 'POXG21', 0.8632401384878545)],
 [],
 [],
 [([[15.512570837685422, 32.11759917275959],
    [207.49189019453658, 2.583568314403899],
    [213.48742916231458, 55.88240082724041],
    [21.508109805463405, 84.4164316855961]],
   '53JGZ-5;',
   0.6257221142358117)],
 [([[-3.664100588675687, 7.503849116986469],
    [40.44554122479882, 2.2623786034324165],
    [42.66410058867569, 27.49615088301353],
    [-2.4455412247988213, 32.73762139656758]],
   'GEv2al',
   0.06542066747688623)],
 [],
 [([[141.3480186333612, 32.10925403674746],
    [246.99990930322195, 47.98095324473122],
    [243.6519813666388, 64.89074596325254],
    [138.00009069677805, 49.01904675526878]],
   'Porschd',
   0.22579402196981668),
  ([[141.3053712883528, 91.1